<a href="https://colab.research.google.com/github/Paulkm2006/novelai_colab_gdrive/blob/main/NovelAILeaks_API_Backend_%EF%BC%88Google_Drive%E7%89%88%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

本项目基于 https://colab.research.google.com/drive/1_Ma71L6uGbtt6UQyA3FjqW2lcZ5Bjck- 魔改，感谢相关大佬（

在此项目的基础上增加了在gd中存储模型，避免每次启动都要下载、解压模型所消耗的大量时间

## 请先“复制到云端硬盘”再使用！！！

第一次运行时运行第0步后即可运行第一步，后期仅需运行第一步

In [ ]:
#@title ### 0. 第一次先运行这个

#@markdown 是否使用团队盘
use_team = False #@param {type: 'boolean'}

#@markdown 盘名称（若不使用请留空）
team_name = ''#@param {type: 'string'}
if use_team:
  if team_name == '':
    print("错误：未指定团队盘")
    raise
  path = '/content/drive/Shareddrives/'+team_name
else:
  path = '/content/drive/MyDrive'

from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
if not os.path.exists(path):
  print("错误：找不到相应团队盘")
  raise

#@markdown 是否使用7GB模型（不推荐，加载慢）
use_latest = False #@param {type: 'boolean'}

os.chdir(path)
!apt install -y -qq aria2
if use_latest:
  !aria2c --summary-interval=5 -x 3 --allow-overwrite=true -Z https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/naifu.tar https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/animefull-latest.tar
  !tar vxf naifu.tar && rm naifu.tar
  !tar vxf animefull-latest.tar -C naifu/models && rm animefull-latest.tar
  !sed -i 's/map_location="cpu"/map_location="cuda"/g' naifu/hydra_node/models.py
else:
  !aria2c --summary-interval=5 -x 3 --allow-overwrite=true -Z https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/naifu.tar
  !tar vxf naifu.tar && rm naifu.tar

os.chdir(path+"/naifu")
!curl -Ls https://github.com/ekzhang/bore/releases/download/v0.4.0/bore-v0.4.0-x86_64-unknown-linux-musl.tar.gz | tar zx
!wget -O cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 && chmod +x cloudflared


In [ ]:
#@title ### 1.enjoy it
#@markdown 是否使用团队盘
use_team = False #@param {type: 'boolean'}

#@markdown 盘名称（若不使用请留空）
team_name = ''#@param {type: 'string'}
if use_team:
  if team_name == '':
    print("错误：未指定团队盘")
    raise
  path = '/content/drive/Shareddrives/'+team_name
else:
  path = '/content/drive/MyDrive'

from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
if not os.path.exists(path):
  print("错误：找不到相应团队盘")
  raise
#@markdown 是否使用7GB模型（不推荐，加载慢）
use_latest = False #@param {type: 'boolean'}

path+="/naifu"
os.chdir(path)
!chmod +x bore
!chmod +x cloudflared
!pip install -r requirements.txt
!pip install --upgrade Pillows
if not use_latest:
  os.environ['DTYPE']='float32'
  os.environ['CLIP_CONTEXTS']='3'
  os.environ['AMP']='1'
  os.environ['MODEL']='stable-diffusion'
  os.environ['DEV']='True'
  os.environ['MODEL_PATH']='models/animefull-final-pruned'
  os.environ['ENABLE_EMA']='1'
  os.environ['VAE_PATH']='models/animevae.pt'
  os.environ['PENULTIMATE']='1'
  os.environ['PYTHONDONTWRITEBYTECODE']='1'
  os.environ['SAVE_FILES']='1'
  !python -m uvicorn --host 0.0.0.0 --port=6969 main:app & ./bore local 6969 --to bore.pub & ./cloudflared tunnel --url localhost:6969
else:
  if not os.path.exists(path+"/models/animefull-latest"):
    raise "错误：未找到7GB模型"
  else:
    os.environ['DTYPE']='float16'
    os.environ['CLIP_CONTEXTS']='3'
    os.environ['AMP']='1'
    os.environ['MODEL']='stable-diffusion'
    os.environ['DEV']='True'
    os.environ['MODEL_PATH']='models/animefull-latest'
    os.environ['ENABLE_EMA']='1'
    os.environ['VAE_PATH']='models/animevae.pt'
    os.environ['PENULTIMATE']='1'
    os.environ['PYTHONDONTWRITEBYTECODE']='1'
    os.environ['SAVE_FILES']='1'
    !python -m uvicorn --host 0.0.0.0 --port=6969 main:app & ./bore local 6969 --to bore.pub & ./cloudflared tunnel --url localhost:6969